# Required libs and Pulling in Pre-trained Model

In [9]:
from transformers import BertTokenizer, TFBertForMaskedLM
from datasets import load_dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForMaskedLM.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


# Loading in dataset from Hugging Face

In [10]:
# load in the dataset
dataset_name = "c4"
task_name = "en"
dataset = load_dataset(dataset_name, task_name, split="validation",streaming=True)

#convert dataset to a list
dataset_list = list(dataset)

# count = 0
# for example in dataset_list:
#     if count >= 1000:
#         break
#     # Process or use the example as needed
#     print(example["text"])  # Or perform any other operation
#     count += 1


# Setting code up to get metrics

In [11]:

# sampleing subset for demonstration purposes
subset_size = 100
data_subset = dataset_list[:min(subset_size, len(dataset_list))]

# extract texts from the subset
texts = [example["text"] for example in data_subset]

max_length = 128

# tokenize the texts
inputs = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

In [16]:
inputs_tuple = tuple(inputs[key] for key in inputs.keys())
predictions = model.predict(inputs_tuple)

4/4 [==============================] - 31s 6s/step


In [17]:
import tensorflow as tf
logits = predictions.logits

# Calculate the mean cross-entropy loss as a measure of perplexity
loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(inputs['input_ids'], logits, from_logits=True))
perplexity = tf.exp(loss)

# Metrics (Loss and Perplexity)

In [18]:
print(f"Loss: {loss.numpy()}")
print(f"perplexity: {perplexity.numpy()}")

Loss: 1.500106692314148
perplexity: 4.4821672439575195
